In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [2]:
spark = (SparkSession.builder.appName("PySparkDFPractice").getOrCreate())

In [3]:
data = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

In [4]:
type(data)

list

In [5]:
schema = StructType([
    StructField("firstname", StringType(), True),
    StructField("middlename", StringType(), True),
    StructField("lastname", StringType(), True),
    StructField("id", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("salary", IntegerType(), True)
])

In [6]:
df = spark.createDataFrame(data=data, schema=schema)
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)



## Working with Structured Operations

### Reading a JSON File

In [7]:
from pyspark.sql.types import ArrayType, FloatType, DateType, BooleanType

In [8]:
persons_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("fav_movies", ArrayType(StringType()), True),
    StructField("salary", FloatType(), True),
    StructField("image_url", StringType(), True),
    StructField("date_of_birth", DateType(), True),
    StructField("active", BooleanType(), True)
])

In [9]:
json_file_path = './data/persons/persons.json'
persons_df = (spark.read.json(json_file_path, persons_schema, multiLine="True"))

In [10]:
persons_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- fav_movies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- salary: float (nullable = true)
 |-- image_url: string (nullable = true)
 |-- date_of_birth: date (nullable = true)
 |-- active: boolean (nullable = true)



In [11]:
persons_df.show(10, truncate=False)

+---+----------+---------+-------------------------------------------------------------+-------+-----------------------------------------------+-------------+------+
|id |first_name|last_name|fav_movies                                                   |salary |image_url                                      |date_of_birth|active|
+---+----------+---------+-------------------------------------------------------------+-------+-----------------------------------------------+-------------+------+
|1  |Drucy     |Poppy    |[I giorni contati]                                           |1463.36|http://dummyimage.com/126x166.png/cc0000/ffffff|1991-02-16   |true  |
|2  |Emelyne   |Blaza    |[Musketeer, The, Topralli]                                   |3006.04|http://dummyimage.com/158x106.bmp/cc0000/ffffff|1991-11-02   |false |
|3  |Max       |Rettie   |[The Forgotten Space, Make It Happen]                        |1422.88|http://dummyimage.com/237x140.jpg/ff4444/ffffff|1990-03-03   |false |
|4  

## Columns and Expressions

In [12]:
from pyspark.sql.functions import col, expr

In [13]:
persons_df.select(col("first_name"), col("last_name"), col("date_of_birth")).show(5)

+----------+---------+-------------+
|first_name|last_name|date_of_birth|
+----------+---------+-------------+
|     Drucy|    Poppy|   1991-02-16|
|   Emelyne|    Blaza|   1991-11-02|
|       Max|   Rettie|   1990-03-03|
|    Ilario|     Kean|   1987-06-09|
|     Toddy|   Drexel|   1992-10-28|
+----------+---------+-------------+
only showing top 5 rows



In [14]:
persons_df.select(expr("first_name"), expr("last_name"), expr("date_of_birth")).show(5)

+----------+---------+-------------+
|first_name|last_name|date_of_birth|
+----------+---------+-------------+
|     Drucy|    Poppy|   1991-02-16|
|   Emelyne|    Blaza|   1991-11-02|
|       Max|   Rettie|   1990-03-03|
|    Ilario|     Kean|   1987-06-09|
|     Toddy|   Drexel|   1992-10-28|
+----------+---------+-------------+
only showing top 5 rows



In [15]:
from pyspark.sql.functions import concat_ws

In [16]:
(persons_df.select(concat_ws(' ', col("first_name"), col("last_name")).alias("full_name"), 
                   col("salary"), 
                   (col("salary") * 0.10 + col("salary")).alias("salary_increase"))).show(10)

+----------------+-------+------------------+
|       full_name| salary|   salary_increase|
+----------------+-------+------------------+
|     Drucy Poppy|1463.36|1609.6959838867188|
|   Emelyne Blaza|3006.04|  3306.64404296875|
|      Max Rettie|1422.88|1565.1680053710938|
|     Ilario Kean|3561.36|3917.4961181640624|
|    Toddy Drexel|4934.87|  5428.35712890625|
| Oswald Petrolli|1153.23| 1268.552978515625|
|   Adrian Clarey|1044.73| 1149.202978515625|
|Dominica Goodnow|1147.76|1262.5360107421875|
|   Emory Slocomb|1082.11|1190.3209838867188|
|   Jeremias Bode|3472.63|  3819.89287109375|
+----------------+-------+------------------+
only showing top 10 rows



In [17]:
(persons_df.select(concat_ws(' ', col("first_name"), col("last_name")).alias("full_name"), 
                   col("salary"), 
                   expr("salary * 0.1 + salary").alias("salary_increase"))).show(10)

+----------------+-------+------------------+
|       full_name| salary|   salary_increase|
+----------------+-------+------------------+
|     Drucy Poppy|1463.36|1609.6959838867188|
|   Emelyne Blaza|3006.04|  3306.64404296875|
|      Max Rettie|1422.88|1565.1680053710938|
|     Ilario Kean|3561.36|3917.4961181640624|
|    Toddy Drexel|4934.87|  5428.35712890625|
| Oswald Petrolli|1153.23| 1268.552978515625|
|   Adrian Clarey|1044.73| 1149.202978515625|
|Dominica Goodnow|1147.76|1262.5360107421875|
|   Emory Slocomb|1082.11|1190.3209838867188|
|   Jeremias Bode|3472.63|  3819.89287109375|
+----------------+-------+------------------+
only showing top 10 rows



## Filter and Where Conditions

In [18]:
persons_df.filter("salary <= 3000").show(10)

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| id|first_name|  last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
|  1|     Drucy|      Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|
|  3|       Max|     Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...|   1990-03-03| false|
|  6|    Oswald|   Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...|   1986-09-02| false|
|  7|    Adrian|     Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...|   1971-08-24| false|
|  8|  Dominica|    Goodnow|    [Hearts Divided]|1147.76|http://dummyimage...|   1973-08-27| false|
|  9|     Emory|    Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...|   1974-06-08|  true|
| 11|   Timothy|     Ervine|[Land of the Lost...|1147.61|http://dummyimage...|   1971-06-02| false|


In [19]:
persons_df.where("salary <= 3000").show(10)

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| id|first_name|  last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
|  1|     Drucy|      Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|
|  3|       Max|     Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...|   1990-03-03| false|
|  6|    Oswald|   Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...|   1986-09-02| false|
|  7|    Adrian|     Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...|   1971-08-24| false|
|  8|  Dominica|    Goodnow|    [Hearts Divided]|1147.76|http://dummyimage...|   1973-08-27| false|
|  9|     Emory|    Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...|   1974-06-08|  true|
| 11|   Timothy|     Ervine|[Land of the Lost...|1147.61|http://dummyimage...|   1971-06-02| false|


In [20]:
persons_df.where((col("salary") <= 3000) & (col("active") == True)).show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
|  1|     Drucy|    Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|
|  9|     Emory|  Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...|   1974-06-08|  true|
| 16|   Margaux| Archbold|[And Now a Word f...|1013.75|http://dummyimage...|   1988-07-29|  true|
| 26|     Clive|      Lax|             [Rabid]|2126.87|http://dummyimage...|   1981-10-26|  true|
| 33|  Sherline|  Primett|   [Jungle Fighters]|2309.39|http://dummyimage...|   1972-07-23|  true|
| 34|     Davis|    Pinks|          [Hounddog]|1337.14|http://dummyimage...|   1989-07-27|  true|
| 37|    Carlen|  Sharply|[Dr. Jekyll and M...|2051.85|http://dummyimage...|   2002-06-01|  true|
| 40|    Jordan|   L

In [21]:
from pyspark.sql.functions import year

In [22]:
persons_df.filter((year("date_of_birth") == 2000) | (year("date_of_birth") == 1989)).show()

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| id|first_name|  last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| 14|   Ambrosi|   Vidineev|[Wall Street: Mon...|4550.88|http://dummyimage...|   1989-07-20|  true|
| 15|    Feodor|Nancekivell|   [Monsoon Wedding]|2218.46|http://dummyimage...|   2000-10-07| false|
| 18|     Alfie|   Hatliffe|     [Lord of Tears]| 3893.1|http://dummyimage...|   1989-06-21|  true|
| 25|     Kelcy|     Wogdon|    [Iron Mask, The]|4512.51|http://dummyimage...|   2000-10-20|  true|
| 32|      Redd|   Akenhead|[Century of the D...| 2470.9|http://dummyimage...|   2000-06-05| false|
| 34|     Davis|      Pinks|          [Hounddog]|1337.14|http://dummyimage...|   1989-07-27|  true|
| 61|    Shanna|    Samples|[Thomas in Love (...| 2703.0|http://dummyimage...|   1989-07-07| false|


In [23]:
from pyspark.sql.functions import array_contains

In [24]:
persons_df.where(array_contains(persons_df.fav_movies, "Land of the Lost")).show()

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| 11|   Timothy|   Ervine|[Land of the Lost...|1147.61|http://dummyimage...|   1971-06-02| false|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+



## Distinct, Drop Duplicates, Order By

In [25]:
from pyspark.sql.functions import count, desc

In [26]:
persons_df.select("active").show(10)

+------+
|active|
+------+
|  true|
| false|
| false|
|  true|
|  true|
| false|
| false|
| false|
|  true|
|  true|
+------+
only showing top 10 rows



In [27]:
persons_df.select("active").distinct().show()

+------+
|active|
+------+
|  true|
| false|
+------+



In [28]:
(persons_df.select(col("first_name"), year(col("date_of_birth")).alias("year"), col("active"))
           .orderBy("year","first_name")).show(10)

+----------+----+------+
|first_name|year|active|
+----------+----+------+
|    Adrian|1971| false|
|   Feodora|1971|  true|
|       Sky|1971| false|
|   Timothy|1971| false|
|    Lucita|1972|  true|
|      Rodi|1972| false|
|  Sherline|1972|  true|
|     Toddy|1972|  true|
|  Dominica|1973| false|
|    Kelila|1973|  true|
+----------+----+------+
only showing top 10 rows



In [29]:
dropped_df = (persons_df.select(col("first_name"), year(col("date_of_birth")).alias("year"), col("active"))
             .dropDuplicates(["year","active"])).orderBy("year","first_name")

In [30]:
dropped_df.show()

+----------+----+------+
|first_name|year|active|
+----------+----+------+
|    Adrian|1971| false|
|   Feodora|1971|  true|
|      Rodi|1972| false|
|  Sherline|1972|  true|
|  Dominica|1973| false|
|    Kelila|1973|  true|
|   Balduin|1974| false|
|     Emory|1974|  true|
|    Janean|1975|  true|
|       Bev|1976|  true|
| Franciska|1976| false|
|     Johny|1977| false|
|    Daveta|1978| false|
|   Guthrie|1978|  true|
|      Maxi|1979| false|
|   Melinda|1979|  true|
|    Carter|1980| false|
|   Loralyn|1980|  true|
|     Clive|1981|  true|
|   Leanora|1981| false|
+----------+----+------+
only showing top 20 rows



In [31]:
(persons_df.select(col("first_name"), year(col("date_of_birth")).alias("year"), col("active"))
           .orderBy("year", ascending=False)).show(10)

+----------+----+------+
|first_name|year|active|
+----------+----+------+
|     Daron|2002|  true|
|    Virgie|2002|  true|
|    Carlen|2002|  true|
|   Lorilee|2002| false|
|    Maxine|2001| false|
|    Feodor|2000| false|
|     Kelcy|2000|  true|
|  Annabell|2000|  true|
|      Redd|2000| false|
|     Jobie|2000| false|
+----------+----+------+
only showing top 10 rows



## Rows and Unions

In [32]:
from pyspark.sql import Row

In [33]:
person_row = Row(101, "Robert","Owens",["Men in Black III", "Home Alone"], 4300.64,"http://someimage.com","1964-08-18", True)

In [34]:
person_rows_list =[Row(101, "Kenny","Bobien",["Men in Black III", "Home Alone"], 4300.64,"http://someimage.com","1964-08-18", True),
                  Row(101, "Sarah","Devine",["Men in Black III", "Home Alone"], 4300.64,"http://someimage.com","1964-08-18", True)]

In [35]:
person_rows_list.append(person_row)

In [36]:
print(person_rows_list)

[<Row(101, 'Kenny', 'Bobien', ['Men in Black III', 'Home Alone'], 4300.64, 'http://someimage.com', '1964-08-18', True)>, <Row(101, 'Sarah', 'Devine', ['Men in Black III', 'Home Alone'], 4300.64, 'http://someimage.com', '1964-08-18', True)>, <Row(101, 'Robert', 'Owens', ['Men in Black III', 'Home Alone'], 4300.64, 'http://someimage.com', '1964-08-18', True)>]


In [37]:
person_row[1]

'Robert'

In [38]:
new_persons_df = spark.createDataFrame(person_rows_list, ["id","first_name","last_name","fav_movies","salary","image_url","date_of_birth","active"])

In [39]:
add_persons_df = persons_df.union(new_persons_df)
add_persons_df.show(10)

+---+----------+---------+--------------------+------------------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies|            salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+------------------+--------------------+-------------+------+
|  1|     Drucy|    Poppy|  [I giorni contati]|1463.3599853515625|http://dummyimage...|   1991-02-16|  true|
|  2|   Emelyne|    Blaza|[Musketeer, The, ...|   3006.0400390625|http://dummyimage...|   1991-11-02| false|
|  3|       Max|   Rettie|[The Forgotten Sp...|1422.8800048828125|http://dummyimage...|   1990-03-03| false|
|  4|    Ilario|     Kean|[Up Close and Per...| 3561.360107421875|http://dummyimage...|   1987-06-09|  true|
|  5|     Toddy|   Drexel|[Walk in the Clou...|   4934.8701171875|http://dummyimage...|   1992-10-28|  true|
|  6|    Oswald| Petrolli|[Wing and the Thi...|  1153.22998046875|http://dummyimage...|   1986-09-02| false|
|  7|    Adrian|   

## Adding, Renaming and Dropping Columns

In [40]:
from pyspark.sql.functions import round

In [41]:
aug_persons_df1 = persons_df.withColumn("salary_increase", expr("salary * 0.10 + salary"))
aug_persons_df1.show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+------------------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|   salary_increase|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+------------------+
|  1|     Drucy|    Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|1609.6959838867188|
|  2|   Emelyne|    Blaza|[Musketeer, The, ...|3006.04|http://dummyimage...|   1991-11-02| false|  3306.64404296875|
|  3|       Max|   Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...|   1990-03-03| false|1565.1680053710938|
|  4|    Ilario|     Kean|[Up Close and Per...|3561.36|http://dummyimage...|   1987-06-09|  true|3917.4961181640624|
|  5|     Toddy|   Drexel|[Walk in the Clou...|4934.87|http://dummyimage...|   1992-10-28|  true|  5428.35712890625|
|  6|    Oswald| Petrolli|[Wing and the Thi...|1153.23|http://du

In [42]:
aug_persons_df1.columns

['id',
 'first_name',
 'last_name',
 'fav_movies',
 'salary',
 'image_url',
 'date_of_birth',
 'active',
 'salary_increase']

In [43]:
aug_persons_df2 = (aug_persons_df1.withColumn("birth_year", year("date_of_birth"))
                   .withColumnRenamed("fav_movies","movies")
                   .withColumn("salary_x10", round(col("salary_increase"), 2))
                   .drop("salary_increase"))

In [44]:
aug_persons_df2.show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+----------+----------+
| id|first_name|last_name|              movies| salary|           image_url|date_of_birth|active|birth_year|salary_x10|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+----------+----------+
|  1|     Drucy|    Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|      1991|    1609.7|
|  2|   Emelyne|    Blaza|[Musketeer, The, ...|3006.04|http://dummyimage...|   1991-11-02| false|      1991|   3306.64|
|  3|       Max|   Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...|   1990-03-03| false|      1990|   1565.17|
|  4|    Ilario|     Kean|[Up Close and Per...|3561.36|http://dummyimage...|   1987-06-09|  true|      1987|    3917.5|
|  5|     Toddy|   Drexel|[Walk in the Clou...|4934.87|http://dummyimage...|   1992-10-28|  true|      1992|   5428.36|
|  6|    Oswald| Petrolli|[Wing and the 

## Working with missing or bad data

In [45]:
bad_movies_list = [Row(None, None, None),
                   Row(None, None, 2020),
                   Row("John Doe", "Awesome Movie", None),
                   Row(None, "Awesome Movie", 2021),
                   Row("Mary Jane", None, 2019),
                   Row("Vikter Duplaix", "Not another teen movie", 2001)]

In [46]:
bad_movies_list

[<Row(None, None, None)>,
 <Row(None, None, 2020)>,
 <Row('John Doe', 'Awesome Movie', None)>,
 <Row(None, 'Awesome Movie', 2021)>,
 <Row('Mary Jane', None, 2019)>,
 <Row('Vikter Duplaix', 'Not another teen movie', 2001)>]

In [47]:
bad_movies_columns =["actor_name","movie_title","produced_year"]

In [48]:
bad_movies_df = spark.createDataFrame(bad_movies_list, schema=bad_movies_columns)

## Woking with User Defined Functions

In [49]:
from pyspark.sql.functions import udf

In [50]:
student_list = [("Joe", 85),("Jane", 90),("Mary", 55)]

In [51]:
students_columns = ["name","score"]

In [52]:
students_df = spark.createDataFrame(student_list, schema=students_columns)

In [53]:
def letterGrade(score:int):
    grade = ''
    if score > 100:
        grade = 'Cheating'
    elif score >= 90:
        grade = 'A'
    elif score >= 80:
        grade = 'B'
    elif score >= 70:
        grade = 'C'
    else:
        grade = 'F'
    return grade    

In [54]:
print(letterGrade(75))

C


In [55]:
letterGradeUDF = udf(letterGrade)

## Aggregations

In [56]:
flight_file = './data/flights/flight-summary.csv'
flight_summary_df = (spark.read.format("csv").option("header","true").option("inferSchema","true").load(flight_file))

In [57]:
flight_summary_df.count()

4693

In [58]:
flight_summary_df.show(10, False)

+-----------+------------------------------------------------+------------+------------+---------+--------------------------------------------+----------------+----------+-----+
|origin_code|origin_airport                                  |origin_city |origin_state|dest_code|dest_airport                                |dest_city       |dest_state|count|
+-----------+------------------------------------------------+------------+------------+---------+--------------------------------------------+----------------+----------+-----+
|BQN        |Rafael Hernández Airport                        |Aguadilla   |PR          |MCO      |Orlando International Airport               |Orlando         |FL        |441  |
|PHL        |Philadelphia International Airport              |Philadelphia|PA          |MCO      |Orlando International Airport               |Orlando         |FL        |4869 |
|MCI        |Kansas City International Airport               |Kansas City |MO          |IAH      |George Bush 

In [59]:
flight_summary_df.printSchema()

root
 |-- origin_code: string (nullable = true)
 |-- origin_airport: string (nullable = true)
 |-- origin_city: string (nullable = true)
 |-- origin_state: string (nullable = true)
 |-- dest_code: string (nullable = true)
 |-- dest_airport: string (nullable = true)
 |-- dest_city: string (nullable = true)
 |-- dest_state: string (nullable = true)
 |-- count: integer (nullable = true)



In [60]:
flight_summary_df = flight_summary_df.withColumnRenamed("count","flight_count")

In [61]:
flight_summary_df.show(10, False)

+-----------+------------------------------------------------+------------+------------+---------+--------------------------------------------+----------------+----------+------------+
|origin_code|origin_airport                                  |origin_city |origin_state|dest_code|dest_airport                                |dest_city       |dest_state|flight_count|
+-----------+------------------------------------------------+------------+------------+---------+--------------------------------------------+----------------+----------+------------+
|BQN        |Rafael Hernández Airport                        |Aguadilla   |PR          |MCO      |Orlando International Airport               |Orlando         |FL        |441         |
|PHL        |Philadelphia International Airport              |Philadelphia|PA          |MCO      |Orlando International Airport               |Orlando         |FL        |4869        |
|MCI        |Kansas City International Airport               |Kansas City |

### Count(col) and Count Distinct

In [62]:
from pyspark.sql.functions import count, countDistinct

In [63]:
flight_summary_df.select(count("origin_airport"), count("dest_airport")).show()

+---------------------+-------------------+
|count(origin_airport)|count(dest_airport)|
+---------------------+-------------------+
|                 4693|               4693|
+---------------------+-------------------+



In [64]:
flight_summary_df.select(countDistinct("origin_airport"), countDistinct("dest_airport"), count('*')).show()

+------------------------------+----------------------------+--------+
|count(DISTINCT origin_airport)|count(DISTINCT dest_airport)|count(1)|
+------------------------------+----------------------------+--------+
|                           322|                         322|    4693|
+------------------------------+----------------------------+--------+



### Min(col), Max(col), Sum(col), sum distinct(col) and avg

In [65]:
from pyspark.sql.functions import min, max, sum, sumDistinct, avg

In [66]:
flight_summary_df.select(min("flight_count"), max("flight_count")).show()

+-----------------+-----------------+
|min(flight_count)|max(flight_count)|
+-----------------+-----------------+
|                1|            13744|
+-----------------+-----------------+



In [67]:
flight_summary_df.select(sum("flight_count")).show()

+-----------------+
|sum(flight_count)|
+-----------------+
|          5332914|
+-----------------+



In [68]:
flight_summary_df.select(sumDistinct("flight_count")).show()

+--------------------------+
|sum(DISTINCT flight_count)|
+--------------------------+
|                   3612257|
+--------------------------+



In [69]:
flight_summary_df.select(avg("flight_count"), sum("flight_count")/count("flight_count")).show()

+------------------+-----------------------------------------+
| avg(flight_count)|(sum(flight_count) / count(flight_count))|
+------------------+-----------------------------------------+
|1136.3549968037503|                       1136.3549968037503|
+------------------+-----------------------------------------+



### Aggregation with grouping

In [70]:
flight_summary_df.groupBy("origin_airport").count().orderBy("count", ascending=False).show(5, False)

+------------------------------------------------+-----+
|origin_airport                                  |count|
+------------------------------------------------+-----+
|Hartsfield-Jackson Atlanta International Airport|169  |
|Chicago O'Hare International Airport            |162  |
|Dallas/Fort Worth International Airport         |148  |
|Denver International Airport                    |139  |
|Minneapolis-Saint Paul International Airport    |120  |
+------------------------------------------------+-----+
only showing top 5 rows



In [71]:
(flight_summary_df.groupBy("origin_airport")
                 .agg(max("flight_count").alias("max_flight_count"))
                 .orderBy("max_flight_count", ascending=False)).show(5, False)

+----------------------------------------------------------------------+----------------+
|origin_airport                                                        |max_flight_count|
+----------------------------------------------------------------------+----------------+
|San Francisco International Airport                                   |13744           |
|Los Angeles International Airport                                     |13457           |
|John F. Kennedy International Airport (New York International Airport)|12016           |
|McCarran International Airport                                        |9715            |
|LaGuardia Airport (Marine Air Terminal)                               |9639            |
+----------------------------------------------------------------------+----------------+
only showing top 5 rows



In [74]:
(flight_summary_df.groupBy("origin_state","origin_city")
                  .count()
                  .where(col("origin_state") == "CA")
                  .orderBy("count", ascending=False)).show(5)

+------------+-------------+-----+
|origin_state|  origin_city|count|
+------------+-------------+-----+
|          CA|San Francisco|   80|
|          CA|  Los Angeles|   80|
|          CA|    San Diego|   47|
|          CA|      Oakland|   35|
|          CA|   Sacramento|   27|
+------------+-------------+-----+
only showing top 5 rows



In [75]:
(flight_summary_df.groupBy("origin_airport")
                  .agg(max("flight_count"),
                       min("flight_count"),
                       sum("flight_count"),
                       count("flight_count"))).show(5, False)

+-------------------------------------------------+-----------------+-----------------+-----------------+-------------------+
|origin_airport                                   |max(flight_count)|min(flight_count)|sum(flight_count)|count(flight_count)|
+-------------------------------------------------+-----------------+-----------------+-----------------+-------------------+
|Melbourne International Airport                  |1332             |1332             |1332             |1                  |
|San Diego International Airport (Lindbergh Field)|6942             |4                |70207            |46                 |
|Eppley Airfield                                  |2083             |1                |16753            |21                 |
|Kahului Airport                                  |8313             |67               |20627            |18                 |
|Austin-Bergstrom International Airport           |4674             |8                |42067            |41           